In [2]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %%
# 1) Setup
import os, re, json
from pathlib import Path
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pdfplumber, fitz, requests

# %%
# 2) Config
DATA_DIR = Path("./data")
JD_FILE = DATA_DIR / "job_description.pdf"
RESUMES_DIR = DATA_DIR / "resumes"
GITHUB_INSIGHTS_FILE = DATA_DIR / "github_insights.json"  # sample file with insights

OPENROUTER_API_KEY = "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"
SENTENCE_MODEL_NAME = "all-MiniLM-L6-v2"
THRESHOLD = 0.4 

# %%
# 3) PDF extraction
def extract_text_from_pdf(filepath):
    text = ""
    try:
        with pdfplumber.open(filepath) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    except:
        try:
            with fitz.open(filepath) as doc:
                for page in doc:
                    text += page.get_text()
        except Exception as e:
            print(f"Error reading {filepath}: {e}")
    return re.sub(r'\s+', ' ', text.strip())

# %%
# 4) Load JD, resumes, and GitHub insights
def load_documents():
    jd_text = extract_text_from_pdf(JD_FILE)
    resumes = {f.name: extract_text_from_pdf(f) for f in RESUMES_DIR.glob('*.pdf')}
    github_insights = {}
    if GITHUB_INSIGHTS_FILE.exists():2
        with open(GITHUB_INSIGHTS_FILE, 'r') as f:
            github_insights = json.load(f)
    return jd_text, resumes, github_insights

# %%
# 5) Extract keywords (optional helper)
def extract_keywords(text):
    words = re.findall(r'\b[a-zA-Z]{3,}\b', text.lower())
    stop = {"the","and","for","with","this","that","from","your","have","are","was","you"}
    words = [w for w in words if w not in stop]
    return list(set(words))

# %%
# 6) Compute similarity
def compute_similarity(jd_text, resumes, github_insights):
    model = SentenceTransformer(SENTENCE_MODEL_NAME)
    jd_emb = model.encode([jd_text])
    results = []

    print("\n🔍 Calculating Similarities...\n")
    for name, text in resumes.items():
        github_text = github_insights.get(name.replace('.pdf',''), '')
        combined = text + ' ' + github_text

        if len(combined.strip()) == 0:
            print(f"⚠️ Skipping empty file: {name}")
            continue

        emb = model.encode([combined])
        sim = cosine_similarity(jd_emb, emb)[0][0]
        print(f"{name}: {sim:.3f}")  # diagnostic print

        if sim >= THRESHOLD:
            results.append((name, sim))

    df = pd.DataFrame(results, columns=['Candidate', 'Similarity'])
    df = df.sort_values('Similarity', ascending=False)
    return df

# %%
# 7) Run pipeline
jd_text, resumes, github_insights = load_documents()

print("JD text length:", len(jd_text))
for name, text in resumes.items():
    print(f"{name} → {len(text)} characters")

df = compute_similarity(jd_text, resumes, github_insights)
print("\nAll Results:\n", df)

# %%
# 8) Filtered Results (>= 0.6)
filtered = df[df['Similarity'] >= THRESHOLD]
print("\nTop Matches (>= 0.6):\n", filtered)

# %%
# 9) Save to CSV
filtered.to_csv('filtered_matches.csv', index=False)
print("\n✅ Results saved to filtered_matches.csv")
